<a href="https://colab.research.google.com/github/emmanuellaignelet/skillup2/blob/main/SKILLUP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Étapes pour installer Flask en utilisant pip

"""
Vérifier l'installation de Python et pip :
Mettre à jour pip :

"""

In [ ]:
python --version
pip --version

#Créer un environnement virtuel (optionnel mais recommandé) :

#Installer Flask :

In [ ]:


python -m pip install --upgrade pip
python -m venv myenv
source myenv/bin/activate  # (ou myenv\Scripts\activate pour Windows)
pip install flask


#Vérifier l'installation de Flask :

In [ ]:
python
>>> import flask
>>> print(flask.__version__)


#Exemple de Code pour Démarrer avec Flask

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


#Pour exécuter cette application, enregistrez le script dans un fichier app.py et lancez-le avec (sous bash) :

In [ ]:
python app.py


Installer IBM Watson SDK pour Python

In [ ]:
pip install ibm-watson

Installer OpenAI SDK

In [ ]:
pip install openai


#Création du back end avec FLASK

#Structure projet FLASK


In [ ]:
chatbot_project/
├── app.py
├── templates/
│   └── index.html
├── static/
│   ├── css/
│   │   └── style.css
│   └── js/
│       └── script.js
└── requirements.txt


In [ ]:
from flask import Flask, render_template, request, jsonify
from ibm_watson import SpeechToTextV1, TextToSpeechV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import openai

app = Flask(__name__)

# IBM Watson API credentials
speech_to_text_authenticator = IAMAuthenticator('your_ibm_speech_to_text_api_key')
speech_to_text = SpeechToTextV1(authenticator=speech_to_text_authenticator)
speech_to_text.set_service_url('your_ibm_speech_to_text_url')

text_to_speech_authenticator = IAMAuthenticator('your_ibm_text_to_speech_api_key')
text_to_speech = TextToSpeechV1(authenticator=text_to_speech_authenticator)
text_to_speech.set_service_url('your_ibm_text_to_speech_url')

# OpenAI GPT-3 API key
openai.api_key = 'your_openai_api_key'

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/api/speech-to-text', methods=['POST'])
def speech_to_text_api():
    audio = request.files['audio']
    response = speech_to_text.recognize(audio=audio, content_type='audio/wav')
    text = response.result['results'][0]['alternatives'][0]['transcript']
    return jsonify({'text': text})

@app.route('/api/generate-response', methods=['POST'])
def generate_response():
    user_input = request.json['text']
    response = openai.Completion.create(
        engine="davinci",
        prompt=user_input,
        max_tokens=150
    )
    chatbot_response = response.choices[0].text.strip()
    return jsonify({'response': chatbot_response})

@app.route('/api/text-to-speech', methods=['POST'])
def text_to_speech_api():
    text = request.json['text']
    response = text_to_speech.synthesize(
        text,
        voice='en-US_AllisonV3Voice',
        accept='audio/wav'
    ).get_result()
    audio = response.content
    return jsonify({'audio': audio})

if __name__ == '__main__':
    app.run(debug=True)


#Text To Speech

#Développement Front End

"""
Création de l'interface utilisateur avec HTML, CSS, et JavaScript
Fichier templates/index.html :
"""

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Chatbot</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>
    <div id="chat-container">
        <div id="chat-box"></div>
        <input type="text" id="user-input" placeholder="Type a message...">
        <button onclick="sendMessage()">Send</button>
        <button onclick="recordAudio()">Record</button>
    </div>
    <script src="{{ url_for('static', filename='js/script.js') }}"></script>
</body>
</html>


"""
Fichier static/css/style.css :
"""

In [ ]:
body {
    font-family: Arial, sans-serif;
    display: flex;
    justify-content: center;
    align-items: center;
    height: 100vh;
    background-color: #f0f0f0;
}
#chat-container {
    width: 300px;
    background: white;
    padding: 10px;
    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
}
#chat-box {
    height: 300px;
    overflow-y: scroll;
    border: 1px solid #ddd;
    margin-bottom: 10px;
}
#user-input {
    width: calc(100% - 70px);
    padding: 5px;
}
button {
    padding: 5px;
    margin-left: 5px;
}


"""
Fichier static/js/script.js :
"""

In [ ]:
async function sendMessage() {
    const userInput = document.getElementById('user-input').value;
    displayMessage(userInput, 'user');
    const response = await fetch('/api/generate-response', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ text: userInput })
    });
    const data = await response.json();
    displayMessage(data.response, 'chatbot');
    textToSpeech(data.response);
}

function displayMessage(message, sender) {
    const chatBox = document.getElementById('chat-box');
    const messageDiv = document.createElement('div');
    messageDiv.classList.add(sender);
    messageDiv.textContent = message;
    chatBox.appendChild(messageDiv);
}

async function recordAudio() {
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    const mediaRecorder = new MediaRecorder(stream);
    mediaRecorder.start();
    mediaRecorder.ondataavailable = async function (e) {
        const formData = new FormData();
        formData.append('audio', e.data, 'recording.wav');
        const response = await fetch('/api/speech-to-text', {
            method: 'POST',
            body: formData
        });
        const data = await response.json();
        document.getElementById('user-input').value = data.text;
    };
    setTimeout(() => mediaRecorder.stop(), 5000); // Record for 5 seconds
}

async function textToSpeech(text) {
    const response = await fetch('/api/text-to-speech', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ text })
    });
    const data = await response.json();
    const audio = new Audio(URL.createObjectURL(new Blob([data.audio])));
    audio.play();
}


4. Exécution du Projet
Lancer l'application :

Exécutez le fichier app.py avec la commande python app.py.
Accédez à http://127.0.0.1:5000/ dans votre navigateur.
Interaction avec le Chatbot :

Utilisez l'interface pour envoyer des messages ou enregistrer des messages vocaux.
Le chatbot comprendra les messages, générera des réponses via GPT-3, et les synthétisera en utilisant IBM Watson Text-to-Speech.

IBM Speech To Text
Replace {apikey} and {url} with your API key and URL.
Modify {path_to_file} to specify the location of the audio-file.flac file.

In [ ]:
curl -X POST -u "apikey:{apikey}" \
--header "Content-Type: audio/flac" \
--data-binary @{path_to_file}audio-file.flac \
"{url}/v1/recognize"


IBM Text To Speech :

In [ ]:
curl -X POST -u "apikey:{apikey}" \
--header "Content-Type: application/json" \
--header "Accept: audio/wav" \
--data "{\"text\":\"hello world\"}" \
--output hello_world.wav \
"{url}/v1/synthesize?voice=en-US_MichaelV3Voice"



"""
Utiliser une voix différente
"""

In [ ]:
curl -X POST -u "apikey:{apikey}" \
--header "Content-Type: application/json" \
--data "{\"text\":\"hello world\"}" \
--output hello_world.ogg \
"{url}/v1/synthesize?voice=en-US_AllisonV3Voice"
